In [1]:
import numpy
from flask import Flask
from flask import request
from keras.layers import Dense,Dropout,Flatten,MaxPooling2D,Convolution2D,Activation
from keras.models import Sequential
from tensorflow.python.keras.backend import set_session
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow 
from io import BytesIO

app = Flask(__name__)

global graph
graph = tensorflow.get_default_graph()
sess = tensorflow.Session()

def larger_model():
        model = Sequential()
        model.add(Convolution2D(32,(3,3),input_shape=(64,64,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Convolution2D(32,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Convolution2D(64,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        #Compiling the CNN
        model.compile(optimizer = 'rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
        return model

#load model
classifire = larger_model()  
set_session(sess)
classifire.load_weights('93_model.h5') 

@app.route("/predict",methods =['POST'])
def predictions(): 
    #preProcess the image
    request_image = BytesIO(request.files["image"].read())
    image = tensorflow.keras.preprocessing.image.load_img(request_image,target_size=(64,64))
    input_arr = tensorflow.keras.preprocessing.image.img_to_array(image)
    input_arr = numpy.array([input_arr])  # Convert single image to a batch.
    with graph.as_default():
                 set_session(sess)                 
                 prediction = classifire.predict_proba(input_arr)[0]    
                 responseValue= "3" 
    if prediction[0] == 1:
                responseValue = "1"
    elif prediction[0] == 0: 
                responseValue = "0"
        
    return str(responseValue)

if __name__ == '__main__':
    app.run(debug=False, port=5001)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\deepk


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
